## Семинар 1 Индекс

## Intro

### чтение файла 
- конструкция __with open__ (recommended)
- конструкция __open + close__

In [211]:
fpath = 'fpath.txt'

# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 
    
#not reccomended  
file = open(txt_fpath, 'r')  
text = file.read()    
file.close() 

### работа с файлами и папками

### os.path  
путь до файла

In [ ]:
import os

# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath('fpath.txt'))

# возвращает имя файла / папки по полному пути до него
print(os.path.basename('/your/path/to/folder/with/fpath.txt'))

# проверить существование директории - True / False
print(os.path.exists('your/path/to/any/folder/'))

### os.listdir  
возвращает список файлов в данной директории

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'
os.listdir(main_dir)

сделаем пути абсолютными, чтобы наш код не зависел от того, где лежит этот файл

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir)]

не забывайте исключать системные директории, такие как .DS_Store

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir) if not '.DS_Store' in fpath]

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'

for root, dirs, files in os.walk(main_dir):
    for name in files:
        print(os.path.join(root, name))

> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [ ]:
list(os.walk(main_dir))

##  Обратный индекс 

Сам по себе обратный индекс не может осуществлять поиск, для этого необходимо добавить к нему определенную метрику. Это не совсем очевидная задача, поэтому немного отложим ее. А сейчас посмотрим, что полезного можно вытащить из индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе. Так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. какой документ является самым большим / маленьким (очень изощренный способ, когда есть _len_)

### __Задача__: 
получите обратный индекс для коллекция документов.    
Перед этим постройте матрицу терм-документ и сделайте функцию булева поиска, которая по запросу будет возвращать 5 релевантных документов.   
В качестве коллекции возьмите сценарий сезонов сериала Друзья. Одна серия - один документ.

[download_friends_corpus](https://yadi.sk/d/k_M7n63A3adGSz)

Этапы:   
    1. получить коллекцию документов
    2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку
    3. получить матрицу терм-документ, написать функцию поиска по ней
    4. получить обратный индекс в виде словаря, где ключ - нормализованное слово, 
    значение - список файлов, в которых это слово встречается
    5. вывести кусочек индекса в виде таблицы 
    6. сделать анализ обратного индекса. Это задание принимается в виде кода и ответов на вопросы

![](Friends/wedding.png)

Напоминание:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, старайтесь этого избегать

In [1]:
import os
main_dir = 'C:\\Users\Maria Tsareva\Desktop\CompLing2018\Friends'
files_list = []
for root, dirs, files in os.walk(main_dir):
    for name in files:
        files_list.append(os.path.join(root, name))


### пройдитесь по всем папкам коллекции и соберите все пути .txt файлов
### _check : в коллекции должно быть 165 файлов


In [2]:
import re
from nltk.tokenize import word_tokenize, wordpunct_tokenize
len(files_list)

165

In [3]:
from nltk.corpus import stopwords

In [4]:
from pymorphy2 import MorphAnalyzer
def first_normalize(text):
    morph = MorphAnalyzer()
    lemmas = []
    clean_lemmas = []
    text = re.sub('!|\#|\$|%|\&|\(|\)|\*|\+|,|-|\.|\/|\:|;|<|=|>|\?|\@|\[|\]|^|_|`|\{|\}|~|«|»|"|…', '', text)
    text = re.sub('(http://|https://|www.)[a-zA-z0-9]*? ', '', text)
    text = re.sub('[0-9]+', '', text)
    text = re.sub('\s+|\n|\t|\ufeff', ' ', text)
    tokens = word_tokenize(text)
    for el in tokens:
        lemmas.append(morph.parse(el)[0].normal_form)
    
    clean_lemmas = [lemma for lemma in lemmas if lemma not in stop_words]
#     clean_lemmas = ' '.join(clean_lemmas)
    return clean_lemmas

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from stop_words import get_stop_words
stop_words = get_stop_words('russian')
word_vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stop_words,
    ngram_range=(1,3),
    tokenizer=first_normalize,
    max_features=10000,
    use_idf=1,
    smooth_idf=1,
    sublinear_tf=1,
    min_df=3,
    max_df=0.9)

In [6]:
texts = []
for file in files_list:
    with open(file, 'r', encoding='utf-8') as f:
        text = f.read().lower()
        texts.append(text)
sum_len = 0
for text in texts:
    sum_len += len(text)
av_len = float(len(texts) / sum_len)
print(av_len)
# matrix = word_vectorizer.fit_transform(texts).toarray()
### постройте матрицу терм-документ
term_doc_matrix = []


### напишите функцию булева поиска по построенной матрице
def boolean_search() -> list:
    """
    Produces a Boolean search according with the term-document matrix
    :return: list of first 5 relevant documents
    """
    return


#запросы 
input_text = [
    'Моника & Фиби & Рэйчел & Чендлер & Джои & Росс',
    '(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джои) & Росс', 
    '(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джои & (НЕ Росс)'
]

8.324235532478644e-05


In [7]:
import numpy as np
matrix.tofile('C://Users/Maria Tsareva/Desktop/CompLing2018/matr.txt', sep='\n')

NameError: name 'matrix' is not defined

In [8]:
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from math import log
from collections import Counter
import string

<img src="pics/inv_index3.svg" alt="Drawing" style="width: 600px;"/>

Совет для построения обратного индекса: 
> В качестве словаря используйте ``` defaultdict ``` из модуля collections   
Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [9]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    russian_stopwords = set(get_stop_words('russian'))
    words = [x.lower().strip(string.punctuation + '»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]
    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [10]:
texts_ = []
for text in texts:
    prepr = preprocessing(text, del_stopwords=True, del_digit=True)
    texts_.append(prepr)

In [15]:
def freq(query, texts_, files_list):
    query = query.split()
    counts = {}
    for text_, file in zip(texts_, files_list):
        # text_ = preprocessing(text, del_stopwords=True, del_digit=True)
        text_len = len(text_)
        for quer in query:
            count = Counter(text_)[quer]
#             print(file.split('\\')[-1])
            counts[file.split('\\')[-1]] = [count, text_len]
    # n = 0
    # for key, value in counts.items():
    #     dl = value[1]
    #     qf = value[0]
#     print(counts)
    return counts

In [11]:
from collections import defaultdict, Counter


In [ ]:
def freq(query, texts_, texts):
    query = query.split()
    counts = {}
    for text_, text in zip(texts_, texts):
        # text_ = preprocessing(text, del_stopwords=True, del_digit=True)
        text_len = len(text_)
        for quer in query:
            count = Counter(text_)[quer]
            counts[text] = [count, text_len]
    # n = 0
    # for key, value in counts.items():
    #     dl = value[1]
    #     qf = value[0]
    # print(counts.items())
    return counts

In [97]:
ind_fr = pd.DataFrame.from_dict(ind, orient='index')
ind_fr = ind_fr.transpose()
ind_fr.head()

,глоукеншир,удивляться,перо,приотовить,овощ,эстера,универ,назло,де,рейнджерс,...,наушник,праздновать,манжета,эмилио,oдно,ебыть,фабутечь,босиком,приближаться,гончарство
0,Friends - 4x24-25 - The One With Ross's Weddin...,Friends - 1x05 - The One With The East German ...,Friends - 1x19 - The One Where The Monkey Gets...,Friends - 4x06 - The One With The Dirty Girl.r...,Friends - 7x08 - The One Where Chandler Doesn'...,Friends - 2x09 - The One With Phoebe's Dad.ru.txt,Friends - 3x03 - The One With The Jam.ru.txt,Friends - 3x19 - The One With The Tiny T-Shirt...,Friends - 2x17 - The One Where Eddie Moves In....,Friends - 4x10 - The One With The Girl From Po...,...,Friends - 1x08 - The One Where Nana Dies Twice...,Friends - 7x10 - The One With The Holiday Arma...,Friends - 2x01 - The One With Ross's New Girlf...,Friends - 4x09 - The One Where They're Going T...,Friends - 7x12 - The One Where They're Up All ...,Friends - 1x09 - The One Where Underdog Gets A...,Friends - 3x16 - The One With The Morning Afte...,Friends - 7x16 - The One With The Truth About ...,Friends - 7x19 - The One With Ross And Monica'...,Friends - 2x22 - The One With The Two Parties....
1,Friends - 4x23-24 - The One With Ross's Weddin...,Friends - 3x09 - The One With The Football.ru.txt,None,None,Friends - 2x21 - The One With The Bullies.ru.txt,None,None,None,Friends - 2x21 - The One With The Bullies.ru.txt,Friends - 5x10 - The One With The Inappropriat...,...,None,Friends - 6x09 - The One Where Ross Got High.r...,None,None,None,None,None,None,Friends - 1x02 - The One With The Sonogram At ...,None
2,None,Friends - 2x21 - The One With The Bullies.ru.txt,None,None,Friends - 1x04 - The One With George Stephanop...,None,None,None,Friends - 4x04 - The One With The Ballroom Dan...,Friends - 3x01 - The One With The Princess Lei...,...,None,Friends - 1x09 - The One Where Underdog Gets A...,None,None,None,None,None,None,None,None
3,None,Friends - 5x09 - The One With Ross's Sandwich....,None,None,Friends - 5x10 - The One With The Inappropriat...,None,None,None,Friends - 1x01 - The One Where Monica Gets A R...,None,...,None,Friends - 7x01 - The One With Monica's Thunder...,None,None,None,None,None,None,None,None
4,None,Friends - 3x20 - The One With The Dollhouse.ru...,None,None,Friends - 2x19 - The One Where Eddie Won't Go....,None,None,None,Friends - 3x12 - The One With All The Jealousy...,None,...,None,Friends - 5x12 - The One With Chandler's Work ...,None,None,None,None,None,None,None,None


In [92]:
import pandas as pd
ind_sort = sorted(ind, key=lambda k: len(ind[k]), reverse=True)
ind_sort = pd.DataFrame(ind_sort)
ind

,0
0,знать
1,думать
2,делать
3,ладный
4,привет
5,хороший
6,говорить
7,большой
8,wwwtvsubtitlesnet
9,правда


С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным?
- какое самым редким?
- какой набор слов есть во всех документах коллекции?

2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 


## Функция ранжирования Okapi BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k_1+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где   
>$f(q_i,D)$ - частота слова $q_i$ в документе $D$ (TF)       
$|D|$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [13]:
### реализуйте эту функцию ранжирования 
from math import log

k1 = 2.0
b = 0.75

def score_BM25(qf, dl, avgdl, k1, b, N, n, counts) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    score = log((N - n + 0.5) / (n + 0.5)) * (k1 + 1) * qf / (qf + k1 * (1 - b + b * dl / avgdl))
    return score

### __Задача__:    
напишите функцию, которая сортирует поисковую выдачу для любого входящего запроса согласно метрике *Okapi BM25*.    
Выведите 10 первых результатов и их скор по запросу **рождественские каникулы**. 

In [16]:
def fin_ok(query, texts, N, av_len):
    q = freq(query, texts_, files_list)
    n = 0
    for key, value in q.items():
        # print(key, value)
        if value[0] != 0:
            n += 1
        dl = value[1]
        qf = value[0]
        rev_score = score_BM25(qf, dl, av_len, k1, b, N, n, q)
        # print(rev_score)
        value.append(rev_score)
        value.remove(value[0])
        value.remove(value[0])
    scores = sorted(q, key=q.get, reverse=True)
    return scores[:10]

N = len(files)
fin = fin_ok('рождественские каникулы', texts, N, av_len)
fin

["Friends - 6x19 - The One With Joey's Fridge.ru.txt",
 'Friends - 6x09 - The One Where Ross Got High.ru.txt',
 'Friends - 6x12 - The One With The Joke.ru.txt',
 'Friends - 7x10 - The One With The Holiday Armadillo.ru.txt',
 'Friends - 1x01 - The One Where Monica Gets A Roommate.ru.txt',
 'Friends - 1x02 - The One With The Sonogram At The End.ru.txt',
 'Friends - 1x03 - The One With The Thumb.ru.txt',
 'Friends - 1x04 - The One With George Stephanopoulos.ru.txt',
 'Friends - 1x05 - The One With The East German Laundry Detergent.ru.txt',
 'Friends - 1x06 - The One With The Butt.ru.txt']